# Extracción de datos de redes sociales 

### 1. Librerías de trabajo

In [ ]:

#API de twitter
import tweepy

#Extracción del objeto request del URL
from urllib2 import urlopen

#Manipulación de datos
import csv
import json as js


#Persistencia de datos
import pickle 


###  2. Función para Conexión  a twitter

In [ ]:
def conectar_twitter(consumer_key,
                     consumer_secret,
                     access_token,
                     access_token_secret):
    auth = tweepy.OAuthHandler(consumer_key,
                               consumer_secret)
    auth.set_access_token(access_token,
                          access_token_secret)
    api = tweepy.API(auth)
    return api

### 3. Función para obtener todos los tweets de un usuario

In [ ]:
def obtener_tweets_usuario(nombre_de_usuario,api):
    todos = []
    nuevos = api.user_timeline(screen_name = nombre_de_usuario,
                               count=200)
    todos.extend(nuevos)
    maximo = todos[-1].id-1
    while len(nuevos)>0:
        print "obteniendo tweets antes de %s" % maximo
        nuevos = api.user_timeline(screen_name = nombre_de_usuario,count=200,max_id=maximo)
        todos.extend(nuevos)
        maximo = todos[-1].id-1
        print "...%s tweets descargados" % len(todos)
    outtweets = [[tweet.text.encode('utf8'),
                    tweet.is_quote_status,
                    tweet.in_reply_to_status_id,
                    tweet.favorite_count,
                    tweet.source,
                    tweet.retweeted,
                    tweet.coordinates,
                    tweet.entities,
                    tweet.in_reply_to_screen_name,
                    tweet.in_reply_to_user_id,
                    tweet.retweet_count,
                    tweet.id_str,
                    tweet.favorited,
                    tweet.user,
                    tweet.geo,
                    tweet.in_reply_to_user_id_str,
                    tweet.lang,
                    tweet.created_at,
                    tweet.in_reply_to_status_id_str,
                    tweet.place] for tweet in todos]
    
    with open('%s_tweets.csv'%nombre_de_usuario ,'wb') as f:
        writer = csv.writer(f)
        writer.writerow(['text',
                    'is_quote_status',
                    'in_reply_to_status_id',
                    'favorite_count',
                    'source',
                    'retweeted',
                    'coordinates',
                    'entities',
                    'in_reply_to_screen_name',
                    'in_reply_to_user_id',
                    'retweet_count',
                    'id_str',
                    'favorited',
                    'user',
                    'geo',
                    'in_reply_to_user_id_str',
                    'lang',
                    'created_at',
                    'in_reply_to_status_id_str',
                    'place'])
        writer.writerows(outtweets)
    f.close()

### 4. Obtener datos crudos de twitter  

In [ ]:
#El archivo llaves_twitter.json contiene las credenciales de twitter
llaves=js.load(open('llaves_twitter.json','r'),encoding='utf8')

In [ ]:
api = conectar_twitter(access_token=llaves['access_token'],access_token_secret=llaves['access_token_secret'],consumer_key=llaves['consumer_key'],consumer_secret=llaves['consumer_secret'])
obtener_tweets_usuario('pictoline',api)

### 5. Obtener Datos Crudos de FB 

In [ ]:
token = js.load(open('llaves_fb.json','rb'))['token']
page_id = 'fcbayern.es'

In [ ]:
url = """https://graph.facebook.com/v2.8/%s/posts/?fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,likes.limit(0).summary(true),reactions.limit(0).summary(true)&limit=100&access_token=%s
""" % (page_id,token)
data = js.load(urlopen(url))

In [ ]:
k=0
pickle.dump(data,open('datosfb/datos_%d.fb'%k,'wb'))
n_post = len(data['data'])
try:
    while "next" in data['paging']:
        print "%d post extraidos" % n_post
        k += 1
        url = data['paging']['next']
        data = js.load(urlopen(url))
        pickle.dump(data,open('datosfb/datos_%d.fb'%k,'wb'))
        n_post += len(data['data'])
except:
    pass